# 1. Carga de Datos

## 1.1 Carga

In [9]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import mode, gmean, hmean

import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.options.display.max_columns = 100

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [12]:
train_ori = pd.read_csv("data/train.csv")
TARGET = 'Approved'
train_ori.head()

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,B001,P,N,S122,G,0.0,NaN,NaN,NaN,NaN,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,NaN,NaN,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,B003,G,Y,S143,B,0.0,92000.0,5.0,NaN,NaN,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,B001,P,Y,S134,B,2500.0,50000.0,2.0,NaN,NaN,10,0


In [11]:
test_ori = pd.read_csv("data/test.csv")
test_ori.head()

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,APPA70109647212,Male,03/06/88,05/07/16,C10028,C,COM0002222,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4
1,APPB10687939341,Male,13/07/81,01/07/16,C10003,A,COM0001784,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7
2,APPC80449411414,Female,19/11/90,01/07/16,C10009,B,COM0045260,B,4.0,1000.0,B002,P,N,S133,B,0.0,NaN,NaN,NaN,NaN,0
3,APPD30665094501,Female,15/10/92,01/07/16,C10005,A,COM0000085,A,3.0,1465.0,B001,P,N,S133,C,0.0,NaN,NaN,NaN,NaN,0
4,APPE80379821637,Male,21/09/88,01/07/16,C10005,A,COM0006422,A,4.0,2340.0,B004,P,Y,S143,B,500.0,10000.0,2.0,NaN,NaN,0


In [14]:
train_ori[TARGET].value_counts()

0    68693
1     1020
Name: Approved, dtype: int64

In [16]:
print(train_ori.shape)
print(test_ori.shape)

(69713, 22)
(30037, 21)


## 1.2 Revisión Inicial

In [17]:
train_ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69713 entries, 0 to 69712
Data columns (total 22 columns):
ID                                     69713 non-null object
Gender                                 69713 non-null object
DOB                                    69698 non-null object
Lead_Creation_Date                     69713 non-null object
City_Code                              68899 non-null object
City_Category                          68899 non-null object
Employer_Code                          65695 non-null object
Employer_Category1                     65695 non-null object
Employer_Category2                     65415 non-null float64
Monthly_Income                         69713 non-null float64
Customer_Existing_Primary_Bank_Code    60322 non-null object
Primary_Bank_Type                      60322 non-null object
Contacted                              69713 non-null object
Source                                 69713 non-null object
Source_Category                    

In [18]:
train_ori.describe()

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
count,65415.000000,6.971300e+04,69662.000000,42004.000000,42004.000000,22276.000000,22276.000000,69713.000000,69713.000000
mean,3.720187,5.622283e+03,360.928751,39429.982859,3.890629,19.213570,1101.466242,3.948446,0.014631
std,0.807374,1.747671e+05,2288.517927,30727.595990,1.167491,5.847136,752.661394,3.819214,0.120073
min,1.000000,0.000000e+00,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000,0.000000
25%,4.000000,1.650000e+03,0.000000,20000.000000,3.000000,15.250000,649.000000,0.000000,0.000000
50%,4.000000,2.500000e+03,0.000000,30000.000000,4.000000,18.000000,941.000000,2.000000,0.000000
75%,4.000000,4.000000e+03,350.000000,50000.000000,5.000000,20.000000,1295.000000,7.000000,0.000000
max,4.000000,3.838384e+07,545436.500000,300000.000000,6.000000,37.000000,13556.000000,10.000000,1.000000


In [103]:
test_ori.describe()

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
count,28342.000000,3.003700e+04,30005.000000,18166.000000,18166.000000,9652.000000,9652.000000,30037.000000
mean,3.728248,3.977139e+03,348.909060,39482.990201,3.903116,19.280537,1094.914836,3.962313
std,0.794883,2.328960e+04,1000.816847,30527.865594,1.150820,5.882246,727.452303,3.821020
min,1.000000,0.000000e+00,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000
25%,4.000000,1.650000e+03,0.000000,20000.000000,3.000000,15.250000,625.000000,0.000000
50%,4.000000,2.500000e+03,0.000000,30000.000000,4.000000,18.000000,946.000000,2.000000
75%,4.000000,4.000000e+03,350.000000,50000.000000,5.000000,20.000000,1291.000000,7.000000
max,4.000000,3.500000e+06,43000.000000,300000.000000,6.000000,37.000000,6979.000000,10.000000


In [362]:
vars_revision_group = ['Gender',
'DOB',
'Lead_Creation_Date',
'City_Code',
'City_Category',
'Employer_Code',
'Employer_Category1',
'Employer_Category2',
'Customer_Existing_Primary_Bank_Code',
'Primary_Bank_Type',
'Contacted',
'Source',
'Source_Category',
'Loan_Period',
'Var1']

# 2. Preparacion de Datos

In [251]:
train = train_ori.copy()
test = test_ori.copy()

In [252]:
cont = 0
for y in vars_revision_group:
    cont = cont + 1
    print("------------------------------------------------------")
    print(str(cont),". Var: ",y, sep = "")
    print(pd.concat([pd.DataFrame(train_ori[:][y].value_counts(dropna=False).index, columns = ['Atributo']),
           pd.DataFrame(train_ori[:][y].value_counts(dropna=False).values, columns = ['Cantidad']),
           pd.DataFrame(100*train_ori[:][y].value_counts(dropna=False).values/len(train_ori), columns = ['%Total'])], axis = 1))

------------------------------------------------------
1. Var: Gender
  Atributo  Cantidad    %Total
0     Male     39949  57.30495
1   Female     29764  42.69505
------------------------------------------------------
2. Var: DOB
       Atributo  Cantidad    %Total
0      11/01/82       253  0.362917
1      04/03/71       183  0.262505
2      03/03/71       121  0.173569
3      03/03/91       103  0.147749
4      03/03/81        88  0.126232
5      03/03/87        83  0.119060
6      03/03/90        78  0.111887
7      03/03/86        77  0.110453
8      03/03/89        75  0.107584
9      01/08/86        67  0.096108
10     03/03/88        59  0.084633
11     03/03/92        59  0.084633
12     01/08/87        57  0.081764
13     11/01/90        53  0.076026
14     03/03/85        52  0.074592
15     02/08/89        50  0.071723
16     31/08/87        49  0.070288
17     31/08/90        48  0.068854
18     31/08/88        47  0.067419
19     01/09/86        47  0.067419
20     15/07/9

  Atributo  Cantidad     %Total
0        Y     45275  64.944845
1        N     24438  35.055155
------------------------------------------------------
12. Var: Source
   Atributo  Cantidad     %Total
0      S122     30941  44.383401
1      S133     23877  34.250427
2      S159      4474   6.417741
3      S143      3480   4.991895
4      S127      1546   2.217664
5      S137      1400   2.008234
6      S134      1048   1.503306
7      S161       621   0.890795
8      S151       569   0.816204
9      S157       507   0.727268
10     S153       403   0.578084
11     S156       250   0.358613
12     S144       243   0.348572
13     S158       167   0.239554
14     S123        61   0.087502
15     S141        46   0.065985
16     S162        31   0.044468
17     S124        17   0.024386
18     S150         7   0.010041
19     S160         7   0.010041
20     S129         3   0.004303
21     S136         3   0.004303
22     S155         3   0.004303
23     S139         2   0.002869
24     S

In [253]:
# Ordenado según % Frecuencia y % Ratio de Conversión
data_imp_temp = train.copy().fillna(-99)
cont = 0
for y in vars_revision_group:
    cont = cont + 1
    print("------------------------------------------------------")
    print(str(cont),". Var: ",y, sep = "")
    print(y)
    print(pd.concat([pd.DataFrame(data_imp_temp[:][y].value_counts().index, columns = ['Atributo']),
           pd.DataFrame(data_imp_temp[:][y].value_counts().values, columns = ['Cantidad']),
           pd.DataFrame(100*data_imp_temp[:][y].value_counts().values/len(data_imp_temp), columns = ['%Total'])], axis = 1))
    temp = pd.DataFrame(np.array(data_imp_temp.groupby(y, as_index = False, axis = 0)[TARGET].mean()), 
                         columns = [y,"% ratio_conv"])
    temp["% ratio_conv"] = temp["% ratio_conv"]*100
    temp = temp.sort_values(by = ["% ratio_conv"], ascending = [False])
    print(temp)

------------------------------------------------------
1. Var: Gender
Gender
  Atributo  Cantidad    %Total
0     Male     39949  57.30495
1   Female     29764  42.69505
   Gender % ratio_conv
1    Male      1.93246
0  Female     0.833221
------------------------------------------------------
2. Var: DOB
DOB
       Atributo  Cantidad    %Total
0      11/01/82       253  0.362917
1      04/03/71       183  0.262505
2      03/03/71       121  0.173569
3      03/03/91       103  0.147749
4      03/03/81        88  0.126232
5      03/03/87        83  0.119060
6      03/03/90        78  0.111887
7      03/03/86        77  0.110453
8      03/03/89        75  0.107584
9      01/08/86        67  0.096108
10     03/03/92        59  0.084633
11     03/03/88        59  0.084633
12     01/08/87        57  0.081764
13     11/01/90        53  0.076026
14     03/03/85        52  0.074592
15     02/08/89        50  0.071723
16     31/08/87        49  0.070288
17     31/08/90        48  0.068854
18    

6. Var: Employer_Code
Employer_Code
         Atributo  Cantidad    %Total
0             -99      4018  5.763631
1      COM0000002       457  0.655545
2      COM0000003       324  0.464763
3      COM0000004       262  0.375827
4      COM0000005       243  0.348572
5      COM0000006       195  0.279718
6      COM0000007       189  0.271112
7      COM0000009       153  0.219471
8      COM0000010       145  0.207996
9      COM0000013       133  0.190782
10     COM0000012       132  0.189348
11     COM0000011       131  0.187913
12     COM0000014       126  0.180741
13     COM0000008       121  0.173569
14     COM0000015       118  0.169265
15     COM0000017       115  0.164962
16     COM0000021       107  0.153486
17     COM0000018       107  0.153486
18     COM0000019       102  0.146314
19     COM0000024        98  0.140576
20     COM0000023        95  0.136273
21     COM0000020        94  0.134839
22     COM0000016        93  0.133404
23     COM0000028        91  0.130535
24     COM0000

28   S162            0
------------------------------------------------------
13. Var: Source_Category
Source_Category
  Atributo  Cantidad     %Total
0        B     29812  42.763903
1        G     26518  38.038816
2        C     11374  16.315465
3        E      1050   1.506175
4        D       497   0.712923
5        F       459   0.658414
6        A         3   0.004303
  Source_Category % ratio_conv
1               B      1.66041
4               E      1.42857
6               G      1.37265
5               F      1.30719
2               C      1.23088
0               A            0
3               D            0
------------------------------------------------------
14. Var: Loan_Period
Loan_Period
   Atributo  Cantidad     %Total
0     -99.0     27709  39.747249
1       5.0     16653  23.887941
2       4.0     12139  17.412821
3       3.0      7059  10.125802
4       2.0      4266   6.119375
5       1.0      1886   2.705378
6       6.0         1   0.001434
   Loan_Period  % ratio_c

ValueError: Shape of passed values is (1, 2), indices imply (2, 2)

In [255]:
train['NAN_COUNTS_temp'] = train.isnull().sum(axis=1)
test['NAN_COUNTS_temp'] = test.isnull().sum(axis=1)

In [256]:
train['NAN_COUNTS'] = np.nan
train['NAN_COUNTS'] = np.nan

In [257]:
train.loc[(train['NAN_COUNTS_temp']==0)|(train['NAN_COUNTS_temp']==1), 'NAN_COUNTS'] = 0
train.loc[(train['NAN_COUNTS_temp']==2)|(train['NAN_COUNTS_temp']==3), 'NAN_COUNTS'] = 1
train.loc[(train['NAN_COUNTS_temp']==4)|(train['NAN_COUNTS_temp']==5), 'NAN_COUNTS'] = 2
train.loc[train['NAN_COUNTS_temp']>=6, 'NAN_COUNTS'] = 3

test.loc[(test['NAN_COUNTS_temp']==0)|(test['NAN_COUNTS_temp']==1), 'NAN_COUNTS'] = 0
test.loc[(test['NAN_COUNTS_temp']==2)|(test['NAN_COUNTS_temp']==3), 'NAN_COUNTS'] = 1
test.loc[(test['NAN_COUNTS_temp']==4)|(test['NAN_COUNTS_temp']==5), 'NAN_COUNTS'] = 2
test.loc[test['NAN_COUNTS_temp']>=6, 'NAN_COUNTS'] = 3

In [329]:
del train['NAN_COUNTS_temp']
del test['NAN_COUNTS_temp']

In [258]:
train['train_test'] = 'train'
test['train_test'] = 'test'
data = pd.concat([train, test], axis = 0).reset_index(drop = True)

### 2.1 Tratamiento Nulos

In [259]:
pd.concat([pd.DataFrame(data.isnull().sum(), columns = ['cantidad']),
           pd.DataFrame(100*data.isnull().sum()/len(data), columns = ['%Total'])], axis = 1)

,cantidad,%Total
Approved,30037,30.112281
City_Category,1128,1.130827
City_Code,1128,1.130827
Contacted,0,0.000000
Customer_Existing_Primary_Bank_Code,13428,13.461654
DOB,18,0.018045
EMI,67822,67.991980
Employer_Category1,5623,5.637093
Employer_Category2,5993,6.008020
Employer_Code,5623,5.637093


In [273]:
temp = pd.concat([pd.DataFrame(data.isnull().sum(), columns = ['cantidad']),
           pd.DataFrame(100*data.isnull().sum()/len(data), columns = ['%Total'])], axis = 1)
temp[temp['cantidad']!=0]

,cantidad,%Total
Approved,30037,30.112281


In [261]:
# City_Category
data['City_Category'].fillna("SIN_INFO", inplace=True)

In [262]:
# City_Code
data['City_Code'].fillna("SIN_INFO", inplace=True)

In [263]:
# Customer_Existing_Primary_Bank_Code
data['Customer_Existing_Primary_Bank_Code'].fillna("B999", inplace=True)

In [264]:
# DOB
data['DOB'].fillna("11/01/82", inplace=True)

In [265]:
# EMI
data['EMI'].fillna(0, inplace=True)

In [266]:
# Interest_Rate
data['Interest_Rate'].fillna(0, inplace=True)

In [267]:
# Employer_Category1
data['Employer_Category1'].fillna('X', inplace=True)

In [268]:
# Employer_Category2
data['Employer_Category2'].fillna(99, inplace=True)

In [269]:
# Employer_Code
data['Employer_Code'].fillna('COM9999999', inplace=True)

In [270]:
# Existing_EMI
data['Existing_EMI'].fillna(data['Existing_EMI'].median(), inplace=True) # median = 0

In [271]:
# Loan_Amount, Loan_Period
data[['Loan_Amount', 'Loan_Period']] = data[['Loan_Amount', 'Loan_Period']].fillna(0)

In [272]:
# Primary_Bank_Type
data['Primary_Bank_Type'].fillna('X', inplace=True)

### Tratamiento de variables fecha

In [319]:
import datetime
from datetime import date
str_format = '%d/%m/%Y'

# DOB_F
temp = pd.DataFrame()
temp['day'],temp['month'],temp['year'] = list(zip(*data['DOB'].apply(lambda x: x.split('/'))))
temp['year'] = temp['year'].apply(lambda x: str(int(x) + 1900))

In [320]:
temp['DOB_F'] = temp.apply(lambda x: x['day']+'/'+x['month']+'/'+x['year'],axis = 1)
data['DOB_F'] = pd.to_datetime(temp['DOB_F'],format = str_format)

In [321]:
# Lead_Creation_Date
temp = pd.DataFrame()
temp['day'],temp['month'],temp['year'] = list(zip(*data['Lead_Creation_Date'].apply(lambda x: x.split('/'))))
temp['year'] = temp['year'].apply(lambda x: str(int(x) + 2000))

In [322]:
temp['Lead_Creation_Date_F'] = temp.apply(lambda x: x['day']+'/'+x['month']+'/'+x['year'],axis = 1)
data['Lead_Creation_Date_F'] = pd.to_datetime(temp['Lead_Creation_Date_F'],format = str_format)

In [323]:
def age(when, on=date(2017,12,31)): # on=None
    if on is None:
        on = datetime.date.today()
    was_earlier = (on.month, on.day) < (when.month, when.day)
    return on.year - when.year - (was_earlier)

In [324]:
data['edad'] = data.apply(lambda x: age(x['DOB_F'],x['Lead_Creation_Date_F']) , axis = 1)

In [325]:
data['codmes_aplica'] = temp.apply(lambda x: int(x['year']+x['month']),axis = 1)

In [326]:
data.head()

,Approved,City_Category,City_Code,Contacted,Customer_Existing_Primary_Bank_Code,DOB,EMI,Employer_Category1,Employer_Category2,Employer_Code,Existing_EMI,Gender,ID,Interest_Rate,Lead_Creation_Date,Loan_Amount,Loan_Period,Monthly_Income,NAN_COUNTS,NAN_COUNTS_temp,Primary_Bank_Type,Source,Source_Category,Var1,train_test,DOB_F,edad,Lead_Creation_Date_F,codmes_aplica
0,0.0,A,C10001,N,B001,23/07/79,0.0,A,4.0,COM0044082,0.0,Female,APPC90493171225,0.00,15/07/16,0.0,0.0,2000.0,2.0,5,P,S122,G,0,train,1979-07-23,36,2016-07-15,201607
1,0.0,A,C10003,Y,B002,07/12/86,953.0,C,1.0,COM0000002,0.0,Male,APPD40611263344,13.25,04/07/16,20000.0,2.0,3500.0,0.0,1,P,S122,G,10,train,1986-12-07,29,2016-07-04,201607
2,0.0,C,C10125,Y,B003,10/12/82,0.0,C,4.0,COM0005267,0.0,Male,APPE70289249423,0.00,19/07/16,45000.0,4.0,2250.0,1.0,3,G,S143,B,0,train,1982-12-10,33,2016-07-19,201607
3,0.0,C,C10477,Y,B003,30/01/89,0.0,A,4.0,COM0004143,0.0,Male,APPF80273865537,0.00,09/07/16,92000.0,5.0,3500.0,1.0,3,G,S143,B,7,train,1989-01-30,27,2016-07-09,201607
4,0.0,A,C10002,Y,B001,19/04/85,0.0,A,4.0,COM0001781,2500.0,Male,APPG60994436641,0.00,20/07/16,50000.0,2.0,10000.0,1.0,3,P,S134,B,10,train,1985-04-19,31,2016-07-20,201607


### 2.2 Revisión Outliers

In [332]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99750 entries, 0 to 99749
Data columns (total 28 columns):
Approved                               69713 non-null float64
City_Category                          99750 non-null object
City_Code                              99750 non-null object
Contacted                              99750 non-null object
Customer_Existing_Primary_Bank_Code    99750 non-null object
DOB                                    99750 non-null object
EMI                                    99750 non-null float64
Employer_Category1                     99750 non-null object
Employer_Category2                     99750 non-null float64
Employer_Code                          99750 non-null object
Existing_EMI                           99750 non-null float64
Gender                                 99750 non-null object
ID                                     99750 non-null object
Interest_Rate                          99750 non-null float64
Lead_Creation_Date              

In [331]:
list(data.columns)

['Approved',
 'City_Category',
 'City_Code',
 'Contacted',
 'Customer_Existing_Primary_Bank_Code',
 'DOB',
 'EMI',
 'Employer_Category1',
 'Employer_Category2',
 'Employer_Code',
 'Existing_EMI',
 'Gender',
 'ID',
 'Interest_Rate',
 'Lead_Creation_Date',
 'Loan_Amount',
 'Loan_Period',
 'Monthly_Income',
 'NAN_COUNTS',
 'Primary_Bank_Type',
 'Source',
 'Source_Category',
 'Var1',
 'train_test',
 'DOB_F',
 'edad',
 'Lead_Creation_Date_F',
 'codmes_aplica']

In [333]:
vars_rev_ouliers = ['EMI',
'Employer_Category2',
'Existing_EMI',
'Interest_Rate',
'Loan_Amount',
'Loan_Period',
'Monthly_Income',
'NAN_COUNTS',
'Var1',
'edad',
'codmes_aplica']

In [334]:
vector_percentiles = [0,1,2.5,5,10,50,90,92.5,95,97.5,99,99.5,100]

In [335]:
cont = 0
for feature in vars_rev_ouliers:
    cont = cont + 1
    print("------------------------------------------------------")
    print(str(cont),". Var: ",feature, sep = "")
    print(pd.concat([pd.DataFrame(vector_percentiles, columns = ['Percentile']),
                     pd.DataFrame(np.nanpercentile(data[[feature]], vector_percentiles), columns = ['Valor'])], axis = 1))

------------------------------------------------------
1. Var: EMI
    Percentile      Valor
0          0.0      0.000
1          1.0      0.000
2          2.5      0.000
3          5.0      0.000
4         10.0      0.000
5         50.0      0.000
6         90.0   1186.000
7         92.5   1339.000
8         95.0   1585.000
9         97.5   2198.275
10        99.0   2983.000
11        99.5   3588.000
12       100.0  13556.000
------------------------------------------------------
2. Var: Employer_Category2
    Percentile  Valor
0          0.0    1.0
1          1.0    1.0
2          2.5    1.0
3          5.0    1.0
4         10.0    3.0
5         50.0    4.0
6         90.0    4.0
7         92.5    4.0
8         95.0   99.0
9         97.5   99.0
10        99.0   99.0
11        99.5   99.0
12       100.0   99.0
------------------------------------------------------
3. Var: Existing_EMI
    Percentile     Valor
0          0.0       0.0
1          1.0       0.0
2          2.5       0.0
3  

In [336]:
data.loc[data['EMI']>3589,'EMI'] = 3599
data.loc[data['Existing_EMI']>5301,'Existing_EMI'] = 5399
data.loc[data['Loan_Amount']>104001,'Loan_Amount'] = 104999
data.loc[data['Monthly_Income']>14001,'Monthly_Income'] = 14999
data.loc[data['edad']>57,'edad'] = 59

In [338]:
data.head()

,Approved,City_Category,City_Code,Contacted,Customer_Existing_Primary_Bank_Code,DOB,EMI,Employer_Category1,Employer_Category2,Employer_Code,Existing_EMI,Gender,ID,Interest_Rate,Lead_Creation_Date,Loan_Amount,Loan_Period,Monthly_Income,NAN_COUNTS,Primary_Bank_Type,Source,Source_Category,Var1,train_test,DOB_F,edad,Lead_Creation_Date_F,codmes_aplica
0,0.0,A,C10001,N,B001,23/07/79,0.0,A,4.0,COM0044082,0.0,Female,APPC90493171225,0.00,15/07/16,0.0,0.0,2000.0,2.0,P,S122,G,0,train,1979-07-23,36,2016-07-15,201607
1,0.0,A,C10003,Y,B002,07/12/86,953.0,C,1.0,COM0000002,0.0,Male,APPD40611263344,13.25,04/07/16,20000.0,2.0,3500.0,0.0,P,S122,G,10,train,1986-12-07,29,2016-07-04,201607
2,0.0,C,C10125,Y,B003,10/12/82,0.0,C,4.0,COM0005267,0.0,Male,APPE70289249423,0.00,19/07/16,45000.0,4.0,2250.0,1.0,G,S143,B,0,train,1982-12-10,33,2016-07-19,201607
3,0.0,C,C10477,Y,B003,30/01/89,0.0,A,4.0,COM0004143,0.0,Male,APPF80273865537,0.00,09/07/16,92000.0,5.0,3500.0,1.0,G,S143,B,7,train,1989-01-30,27,2016-07-09,201607
4,0.0,A,C10002,Y,B001,19/04/85,0.0,A,4.0,COM0001781,2500.0,Male,APPG60994436641,0.00,20/07/16,50000.0,2.0,10000.0,1.0,P,S134,B,10,train,1985-04-19,31,2016-07-20,201607


In [339]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99750 entries, 0 to 99749
Data columns (total 28 columns):
Approved                               69713 non-null float64
City_Category                          99750 non-null object
City_Code                              99750 non-null object
Contacted                              99750 non-null object
Customer_Existing_Primary_Bank_Code    99750 non-null object
DOB                                    99750 non-null object
EMI                                    99750 non-null float64
Employer_Category1                     99750 non-null object
Employer_Category2                     99750 non-null float64
Employer_Code                          99750 non-null object
Existing_EMI                           99750 non-null float64
Gender                                 99750 non-null object
ID                                     99750 non-null object
Interest_Rate                          99750 non-null float64
Lead_Creation_Date              

### 2.3 Reemplazar valores

In [363]:
# Ordenado según % Frecuencia y % Ratio de Conversión
data_imp_temp = data[data['train_test']=='train'].copy()
cont = 0
for y in vars_revision_group:
    cont = cont + 1
    print("------------------------------------------------------")
    print(str(cont),". Var: ",y, sep = "")
    print(y)
    print(pd.concat([pd.DataFrame(data_imp_temp[:][y].value_counts().index, columns = ['Atributo']),
           pd.DataFrame(data_imp_temp[:][y].value_counts().values, columns = ['Cantidad']),
           pd.DataFrame(100*data_imp_temp[:][y].value_counts().values/len(data_imp_temp), columns = ['%Total'])], axis = 1))
    temp = pd.DataFrame(np.array(data_imp_temp.groupby(y, as_index = False, axis = 0)[TARGET].mean()), 
                         columns = [y,"% ratio_conv"])
    temp["% ratio_conv"] = temp["% ratio_conv"]*100
    temp = temp.sort_values(by = ["% ratio_conv"], ascending = [False])
    print(temp)

------------------------------------------------------
1. Var: Gender
Gender
  Atributo  Cantidad    %Total
0     Male     39949  57.30495
1   Female     29764  42.69505
   Gender % ratio_conv
1    Male      1.93246
0  Female     0.833221
------------------------------------------------------
2. Var: DOB
DOB
       Atributo  Cantidad    %Total
0      11/01/82       268  0.384433
1      04/03/71       183  0.262505
2      03/03/71       121  0.173569
3      03/03/91       103  0.147749
4      03/03/81        88  0.126232
5      03/03/87        83  0.119060
6      03/03/90        78  0.111887
7      03/03/86        77  0.110453
8      03/03/89        75  0.107584
9      01/08/86        67  0.096108
10     03/03/92        59  0.084633
11     03/03/88        59  0.084633
12     01/08/87        57  0.081764
13     11/01/90        53  0.076026
14     03/03/85        52  0.074592
15     02/08/89        50  0.071723
16     31/08/87        49  0.070288
17     31/08/90        48  0.068854
18    

------------------------------------------------------
6. Var: Employer_Code
Employer_Code
         Atributo  Cantidad    %Total
0      COM9999999      4018  5.763631
1      COM0000002       457  0.655545
2      COM0000003       324  0.464763
3      COM0000004       262  0.375827
4      COM0000005       243  0.348572
5      COM0000006       195  0.279718
6      COM0000007       189  0.271112
7      COM0000009       153  0.219471
8      COM0000010       145  0.207996
9      COM0000013       133  0.190782
10     COM0000012       132  0.189348
11     COM0000011       131  0.187913
12     COM0000014       126  0.180741
13     COM0000008       121  0.173569
14     COM0000015       118  0.169265
15     COM0000017       115  0.164962
16     COM0000021       107  0.153486
17     COM0000018       107  0.153486
18     COM0000019       102  0.146314
19     COM0000024        98  0.140576
20     COM0000023        95  0.136273
21     COM0000020        94  0.134839
22     COM0000016        93  0.1334

------------------------------------------------------
13. Var: Source_Category
Source_Category
  Atributo  Cantidad     %Total
0        B     29812  42.763903
1        G     26518  38.038816
2        C     11374  16.315465
3        E      1050   1.506175
4        D       497   0.712923
5        F       459   0.658414
6        A         3   0.004303
  Source_Category % ratio_conv
1               B      1.66041
4               E      1.42857
6               G      1.37265
5               F      1.30719
2               C      1.23088
0               A            0
3               D            0
------------------------------------------------------
14. Var: Loan_Period
Loan_Period
   Atributo  Cantidad     %Total
0       0.0     27709  39.747249
1       5.0     16653  23.887941
2       4.0     12139  17.412821
3       3.0      7059  10.125802
4       2.0      4266   6.119375
5       1.0      1886   2.705378
6       6.0         1   0.001434
   Loan_Period  % ratio_conv
1          1.0     

In [411]:
data2 = data.copy()

In [412]:
del(data2['DOB'])
del(data2['ID'])
del(data2['Lead_Creation_Date'])
del(data2['DOB_F'])
del(data2['Lead_Creation_Date_F'])

In [413]:
# Gender
dicc_temp = {'Male':1 , 'Female':0}
data2['Gender'] = data2[:]['Gender'].map(dicc_temp)

In [414]:
# City_Category
temp_dummies = pd.get_dummies(data2[:]['City_Category'], prefix = 'City_Category')
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2['City_Category'])

In [415]:
# City_Code
feature = 'City_Code'
data2.loc[(data2[feature] != 'C10001') &
          (data2[feature] != 'C10002') &
          (data2[feature] != 'C10003') &
          (data2[feature] != 'C10004') &
          (data2[feature] != 'C10005') &
          (data2[feature] != 'C10006') &
          (data2[feature] != 'C10007') &
          (data2[feature] != 'C10008'), feature] = 'OTROS'

temp_dummies = pd.get_dummies(data2[:][feature], prefix = feature)
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2[feature])

In [416]:
# Employer_Code
data2['Employer_Code_FlagNulo'] = 0
data2.loc[data2['Employer_Code']=='COM9999999','Employer_Code_FlagNulo'] = 1

del(data2['Employer_Code'])

In [417]:
# Employer_Category1
temp_dummies = pd.get_dummies(data2[:]['Employer_Category1'], prefix = 'Employer_Category1')
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2['Employer_Category1'])

In [418]:
# City_Code
feature = 'Customer_Existing_Primary_Bank_Code'
data2.loc[(data2[feature] != 'B001') &
          (data2[feature] != 'B002') &
          (data2[feature] != 'B003') &
          (data2[feature] != 'B999') &
          (data2[feature] != 'B004') &
          (data2[feature] != 'B005') &
          (data2[feature] != 'B006'), feature] = 'OTROS'

temp_dummies = pd.get_dummies(data2[:][feature], prefix = feature)
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2[feature])

In [419]:
# Primary_Bank_Type
temp_dummies = pd.get_dummies(data2[:]['Primary_Bank_Type'], prefix = 'Primary_Bank_Type')
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2['Primary_Bank_Type'])

In [420]:
# Contacted
dicc_temp = {'Y':1 , 'N':0}
data2['Contacted'] = data2[:]['Contacted'].map(dicc_temp)

In [421]:
# City_Code
feature = 'Source'
data2.loc[(data2[feature] != 'S122') &
          (data2[feature] != 'S133') &
          (data2[feature] != 'S159') &
          (data2[feature] != 'S143') &
          (data2[feature] != 'S127') &
          (data2[feature] != 'S137'), feature] = 'OTROS'

temp_dummies = pd.get_dummies(data2[:][feature], prefix = feature)
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2[feature])

In [422]:
# Source_Category
feature = 'Source_Category'
data2.loc[(data2[feature] != 'B') &
          (data2[feature] != 'G') &
          (data2[feature] != 'C') &
          (data2[feature] != 'E'), feature] = 'OTROS'

temp_dummies = pd.get_dummies(data2[:][feature], prefix = feature)
data2 = pd.concat([data2, temp_dummies], axis = 1)

del(data2[feature])

In [423]:
data2.head()

,Approved,Contacted,EMI,Employer_Category2,Existing_EMI,Gender,Interest_Rate,Loan_Amount,Loan_Period,Monthly_Income,NAN_COUNTS,Var1,train_test,edad,codmes_aplica,City_Category_A,City_Category_B,City_Category_C,City_Category_SIN_INFO,City_Code_C10001,City_Code_C10002,City_Code_C10003,City_Code_C10004,City_Code_C10005,City_Code_C10006,City_Code_C10007,City_Code_C10008,City_Code_OTROS,Employer_Code_FlagNulo,Employer_Category1_A,Employer_Category1_B,Employer_Category1_C,Employer_Category1_X,Customer_Existing_Primary_Bank_Code_B001,Customer_Existing_Primary_Bank_Code_B002,Customer_Existing_Primary_Bank_Code_B003,Customer_Existing_Primary_Bank_Code_B004,Customer_Existing_Primary_Bank_Code_B005,Customer_Existing_Primary_Bank_Code_B006,Customer_Existing_Primary_Bank_Code_B999,Customer_Existing_Primary_Bank_Code_OTROS,Primary_Bank_Type_G,Primary_Bank_Type_P,Primary_Bank_Type_X,Source_OTROS,Source_S122,Source_S127,Source_S133,Source_S137,Source_S143,Source_S159,Source_Category_B,Source_Category_C,Source_Category_E,Source_Category_G,Source_Category_OTROS
0,0.0,0,0.0,4.0,0.0,0,0.00,0.0,0.0,2000.0,2.0,0,train,36,201607,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0.0,1,953.0,1.0,0.0,1,13.25,20000.0,2.0,3500.0,0.0,10,train,29,201607,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
2,0.0,1,0.0,4.0,0.0,1,0.00,45000.0,4.0,2250.0,1.0,0,train,33,201607,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
3,0.0,1,0.0,4.0,0.0,1,0.00,92000.0,5.0,3500.0,1.0,7,train,27,201607,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
4,0.0,1,0.0,4.0,2500.0,1,0.00,50000.0,2.0,10000.0,1.0,10,train,31,201607,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
codmes_aplica

In [424]:
data_imp_temp = data2[data2['train_test']=='train'].copy()
cont = 0
for y in ["codmes_aplica"]:
    cont = cont + 1
    print("------------------------------------------------------")
    print(str(cont),". Var: ",y, sep = "")
    print(y)
    print(pd.concat([pd.DataFrame(data_imp_temp[:][y].value_counts().index, columns = ['Atributo']),
           pd.DataFrame(data_imp_temp[:][y].value_counts().values, columns = ['Cantidad']),
           pd.DataFrame(100*data_imp_temp[:][y].value_counts().values/len(data_imp_temp), columns = ['%Total'])], axis = 1))
    temp = pd.DataFrame(np.array(data_imp_temp.groupby(y, as_index = False, axis = 0)[TARGET].mean()), 
                         columns = [y,"% ratio_conv"])
    temp["% ratio_conv"] = temp["% ratio_conv"]*100
    temp = temp.sort_values(by = ["% ratio_conv"], ascending = [False])
    print(temp)

------------------------------------------------------
1. Var: codmes_aplica
codmes_aplica
   Atributo  Cantidad     %Total
0    201609     25733  36.912771
1    201608     22404  32.137478
2    201607     21576  30.949751
   codmes_aplica  % ratio_conv
1       201608.0      1.624710
0       201607.0      1.436782
2       201609.0      1.344577


In [425]:
dataset = data2.copy()

In [442]:
train = dataset[dataset['train_test'] == 'train'].reset_index(drop = True)
subm = dataset[dataset['train_test'] == 'test'].reset_index(drop = True)

del(train['train_test'])
del(subm['train_test'])

## 3. Modelamiento

## 3.1 Seleccionar variables predictoras

In [427]:
list(dataset.columns)

['Approved',
 'Contacted',
 'EMI',
 'Employer_Category2',
 'Existing_EMI',
 'Gender',
 'Interest_Rate',
 'Loan_Amount',
 'Loan_Period',
 'Monthly_Income',
 'NAN_COUNTS',
 'Var1',
 'train_test',
 'edad',
 'codmes_aplica',
 'City_Category_A',
 'City_Category_B',
 'City_Category_C',
 'City_Category_SIN_INFO',
 'City_Code_C10001',
 'City_Code_C10002',
 'City_Code_C10003',
 'City_Code_C10004',
 'City_Code_C10005',
 'City_Code_C10006',
 'City_Code_C10007',
 'City_Code_C10008',
 'City_Code_OTROS',
 'Employer_Code_FlagNulo',
 'Employer_Category1_A',
 'Employer_Category1_B',
 'Employer_Category1_C',
 'Employer_Category1_X',
 'Customer_Existing_Primary_Bank_Code_B001',
 'Customer_Existing_Primary_Bank_Code_B002',
 'Customer_Existing_Primary_Bank_Code_B003',
 'Customer_Existing_Primary_Bank_Code_B004',
 'Customer_Existing_Primary_Bank_Code_B005',
 'Customer_Existing_Primary_Bank_Code_B006',
 'Customer_Existing_Primary_Bank_Code_B999',
 'Customer_Existing_Primary_Bank_Code_OTROS',
 'Primary_Bank_T

In [443]:
# Dataset vars
features_to_model = ['Contacted',
 'EMI',
 'Employer_Category2',
 'Existing_EMI',
 'Gender',
 'Interest_Rate',
 'Loan_Amount',
 'Loan_Period',
 'Monthly_Income',
 'NAN_COUNTS',
 'Var1',
 'edad',
 'codmes_aplica',
 'City_Category_A',
 'City_Category_B',
 'City_Category_C',
 'City_Category_SIN_INFO',
 'City_Code_C10001',
 'City_Code_C10002',
 'City_Code_C10003',
 'City_Code_C10004',
 'City_Code_C10005',
 'City_Code_C10006',
 'City_Code_C10007',
 'City_Code_C10008',
 'City_Code_OTROS',
 'Employer_Code_FlagNulo',
 'Employer_Category1_A',
 'Employer_Category1_B',
 'Employer_Category1_C',
 'Employer_Category1_X',
 'Customer_Existing_Primary_Bank_Code_B001',
 'Customer_Existing_Primary_Bank_Code_B002',
 'Customer_Existing_Primary_Bank_Code_B003',
 'Customer_Existing_Primary_Bank_Code_B004',
 'Customer_Existing_Primary_Bank_Code_B005',
 'Customer_Existing_Primary_Bank_Code_B006',
 'Customer_Existing_Primary_Bank_Code_B999',
 'Customer_Existing_Primary_Bank_Code_OTROS',
 'Primary_Bank_Type_G',
 'Primary_Bank_Type_P',
 'Primary_Bank_Type_X',
 'Source_OTROS',
 'Source_S122',
 'Source_S127',
 'Source_S133',
 'Source_S137',
 'Source_S143',
 'Source_S159',
 'Source_Category_B',
 'Source_Category_C',
 'Source_Category_E',
 'Source_Category_G',
 'Source_Category_OTROS']

In [444]:
# Predictoras y target
X = train[features_to_model]
y = train[TARGET]

X_subm = subm[features_to_model]

In [454]:
X_train = X.copy()
y_train = y.copy()

X_train_cv  = X.copy()
y_train_cv  = y.copy()

In [446]:
print("train: ", X.shape,", summit: ", X_subm.shape)

train:  (69713, 54) , summit:  (30037, 54)


## 3.2 Modelos

### A. RANDOM FOREST

In [449]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, random_state = 321, n_jobs = 4, max_depth = 5, min_samples_leaf = 20)
                                    #, max_features = 15 )

In [458]:
# Cros Validation
from sklearn.base import clone
from sklearn import cross_validation
from sklearn.metrics import roc_auc_score
k = 5 # n folds
metric = 'roc_auc'
cv = cross_validation.KFold(len(X_train_cv), n_folds=k, random_state = 100, shuffle = True)

results_train = []
results_test = []
list_models = [] 

i = 0
for traincv, testcv in cv:
        model = clone(classifier)
        model = model.fit(X_train_cv.values[traincv], y_train_cv[traincv])
        probas_train = model.predict_proba(X_train_cv.values[traincv])[:,1]
        probas_test = model.predict_proba(X_train_cv.values[testcv])[:,1]
        metric_train = roc_auc_score(y_train_cv[traincv],probas_train)
        metric_test = roc_auc_score(y_train_cv[testcv],probas_test)
        results_train.append(metric_train)
        results_test.append(metric_test)
        list_models.append(model)
        print("Ok - ",str(i),": ", str(round(metric_train,6)), str(round(metric_test,6)))
        print("Mean Target: ", str(round(y_train_cv[traincv].mean(),4)), str(round(y_train_cv[testcv].mean(),4)))

        i = i + 1

result_final_train = np.mean(results_train)
result_final_test = np.mean(results_test)
print("\nFinal - ",str(i),": ", str(round(result_final_train,4)), str(round(result_final_test,4)))

print("Listo!\n")

Ok -  0 :  0.852705 0.823995
Mean Target:  0.0148 0.0141
Ok -  1 :  0.845535 0.861499
Mean Target:  0.0149 0.0136
Ok -  2 :  0.850993 0.832397
Mean Target:  0.0142 0.0162
Ok -  3 :  0.852423 0.822699
Mean Target:  0.0145 0.0151
Ok -  4 :  0.856531 0.805779
Mean Target:  0.0148 0.0141

Final -  5 :  0.8516 0.8293
Listo!



In [459]:
model_rf = classifier.fit(X_train_cv, y_train)
y_pred_proba = model_rf.predict_proba(X_train) 

# Cálculo del AUC-ROC
from sklearn import metrics as mt
auc = mt.roc_auc_score(y_train.astype('int'),y_pred_proba[:,1])
auc

0.84805155390151632

In [195]:
features = pd.DataFrame()
features['feature'] = X_train.columns
features['importance'] = model_rf.feature_importances_

In [196]:
ftrs = features.sort_values(by = ['importance','feature'],ascending=False)
ftrs = ftrs.reset_index(drop = True)
ftrs

,feature,importance
0,ANTIGUEDAD,1.048812e-01
1,RNK_ANTIGUEDAD,9.825517e-02
2,RANG_INGRESO,9.645583e-02
3,RANG_SDO_PASIVO_MENOS0,8.146840e-02
4,RNK_RANG_SDO_PASIVO_MENOS0,8.064433e-02
5,FLG_SDO_OTSSFF_ULT3M,3.989721e-02
6,NAN_COUNTS,2.667484e-02
7,FLG_SDO_OTSSFF_MENOS0,2.404307e-02
8,EDAD,2.175146e-02
9,NRO_ACCES_CANALES_VARS_ULT6M,1.847743e-02


In [197]:
ftrs.to_excel("features/importancia_param_rf_v01.xlsx", index = False)

## B. LightGBM

In [460]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
#lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [557]:
num_boost_round = 10000

lgb_params = {}

lgb_params['learning_rate'] = 0.01
lgb_params['n_estimators'] = 200 #2000
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.7
lgb_params['subsample_freq'] = 20 # 20
lgb_params['colsample_bytree'] = 0.7   
lgb_params['min_child_samples'] = 20
lgb_params['seed'] = 123 #321
lgb_params['objective'] = 'binary'
lgb_params['max_depth'] = 4

#lgb_params['min_data_in_leaf'] = 50

lgb_params['num_leaves'] = 70

lgb_params['metric'] = 'binary_logloss'
lgb_params['training_metric'] = True

In [558]:
# Cros Validation
from sklearn.base import clone
from sklearn import cross_validation
from sklearn.metrics import roc_auc_score

metric = 'roc_auc'
k = 5 
cv = cross_validation.KFold(len(X_train_cv), n_folds=k, random_state = 0, shuffle = True)

results_train = []
results_test = []
list_models = [] 

results_subm = pd.DataFrame()

i = 0
for traincv, testcv in cv:
        lgb_train = lgb.Dataset(X_train_cv.values[traincv], y_train_cv[traincv])
        lgb_eval = lgb.Dataset(X_train_cv.values[testcv], y_train_cv[testcv], reference=lgb_train)
        model = lgb.train(lgb_params,
                          lgb_train,
                          num_boost_round=num_boost_round,
                          valid_sets = lgb_eval,
                          early_stopping_rounds = 50,
                          verbose_eval = 0)
        probas_train = model.predict(X_train_cv.values[traincv], num_iteration=model.best_iteration)
        probas_test = model.predict(X_train_cv.values[testcv], num_iteration=model.best_iteration)
        metric_train = roc_auc_score(y_train_cv[traincv],probas_train)
        metric_test = roc_auc_score(y_train_cv[testcv],probas_test)
        
        results_train.append(metric_train)
        results_test.append(metric_test)
        list_models.append(model)
        print("Ok - ",str(i),": ", str(round(metric_train,6)), str(round(metric_test,6)))
        print("Mean Target: ", str(round(y_train_cv[traincv].mean(),4)), str(round(y_train_cv[testcv].mean(),4)))
        
        results_subm[str(i)] = model.predict(X_subm, num_iteration=model.best_iteration)
        
        i = i + 1

print("Listo!")
result_final_train = np.mean(results_train)
result_final_test = np.mean(results_test)

### TRAIN
print ("auc - train: ", result_final_train) 
### TEST
print ("auc - test : ", result_final_test) 

print("Diff:        ", round(result_final_train-result_final_test,4))

Ok -  0 :  0.851888 0.828709
Mean Target:  0.0148 0.0138
Ok -  1 :  0.879858 0.836231
Mean Target:  0.0149 0.0137
Ok -  2 :  0.883012 0.84952
Mean Target:  0.0146 0.0146
Ok -  3 :  0.894782 0.849944
Mean Target:  0.0145 0.015
Ok -  4 :  0.895881 0.858694
Mean Target:  0.0143 0.016
Listo!
auc - train:  0.881084291895
auc - test :  0.844619465194
Diff:         0.0365


In [ ]:
auc - train:  0.892228960435
auc - test :  0.847996196235
Diff:         0.0442

Public LB : 0.8489357904
    
# Parametros
num_boost_round = 10000

lgb_params = {}

lgb_params['learning_rate'] = 0.01
lgb_params['n_estimators'] = 200 #2000
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.7
lgb_params['subsample_freq'] = 20 # 20
lgb_params['colsample_bytree'] = 0.7   
lgb_params['min_child_samples'] = 20
lgb_params['seed'] = 123 #321
lgb_params['objective'] = 'binary'
lgb_params['max_depth'] = 4

#lgb_params['min_data_in_leaf'] = 50

lgb_params['num_leaves'] = 50

lgb_params['metric'] = 'binary_logloss'
lgb_params['training_metric'] = True

,0,1,2,3,4,TARGET
0,0.006147,0.008283,0.007345,0.008695,0.010087,0.008112
1,0.015815,0.023514,0.016304,0.014804,0.014354,0.016958
2,0.000141,0.000817,0.000564,0.000214,0.000168,0.000381
3,0.000120,0.000588,0.000468,0.000167,0.000093,0.000287
4,0.004016,0.004425,0.003472,0.004228,0.003689,0.003966


# Submission

In [565]:
results_subm["TARGET"] = results_subm['4']#results_subm.apply(np.mean,axis=1)
results_subm.head()

,0,1,2,3,4,TARGET
0,0.073405,0.008283,0.007345,0.008695,0.010087,0.010087
1,0.101351,0.023514,0.016304,0.014804,0.014354,0.014354
2,0.068350,0.000817,0.000564,0.000214,0.000168,0.000168
3,0.068174,0.000588,0.000468,0.000167,0.000093,0.000093
4,0.071904,0.004425,0.003472,0.004228,0.003689,0.003689


In [566]:
y_pred_summit = results_subm["TARGET"].values
summit_final = pd.concat( [test_ori.ix[:,'ID'] , pd.DataFrame(y_pred_summit,columns = ['Approved'] )], axis = 1)
summit_final.head()

,ID,Approved
0,APPA70109647212,0.010087
1,APPB10687939341,0.014354
2,APPC80449411414,0.000168
3,APPD30665094501,0.000093
4,APPE80379821637,0.003689


In [567]:
summit_final.to_csv("submissions/Summit_lgbmgb_03_54vars_prueba.csv", index = False)

In [ ]:
8460, 8448, 8493